# Main Notebook

## Summary

This notebook will do following steps:
 -  Pull validation data
 -  Load CTR predictions from CSV for X different models
 -  Load BaseBid Predictions from pickels for X different models
 -  For X models combine (in Y different ways) the above two steps to build X*Y BidPrices for each impression
 -  Hold auction for each impression one by one
 -  Summary Scores for each of X*Y models 

# Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint    
import seaborn as sns
from score_models import Scorer

%matplotlib inline

# Pull Validation Data

In [2]:
path = "C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/"
#test_df = pd.read_csv(path+'test.csv')
#train_df = pd.read_csv(path+'train.csv')
validation_df = pd.read_csv(path+'validation.csv')

In [3]:
#print(validation_df.head())
avgCTR =validation_df.loc[validation_df["click"] == 1].shape[0]/(validation_df.shape[0])
print(avgCTR)

0.0006646376573167722


# Load CTR Predictions

Need a function that will combine multiple dataframes into one

In [42]:
path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/Predictions/click_predictions_achilleas/'
teamACTRData = pd.read_csv(path+'lr_pCTR.csv')
teamACTR=teamACTRData[['click_proba']]
teamACTR = teamACTR.rename(index=str, columns={"click_proba": "pCTR"})
print(teamACTR.head(2))

path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/Predictions/click_predictions_dwane/'
teamBCTRData = pd.read_csv(path+'click_predictions_narrow.csv')
teamBCTR=teamBCTRData[['pCTR']]
#teamBCTR = teamBCTR.rename(index=str, columns={"click_proba": "pCTR"})
print(teamBCTR.head(2))

       pCTR
0  0.470948
1  0.393028
   pCTR
0   0.0
1   0.0


In [43]:
#resultCTR=teamAdf
#resultCTR["teamB_CTR"]=teamBdf["CTR"]
#result = pd.merge(teamAdf, teamBdf, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
#print(resultCTR.head())

# Load Basebid Predictions

Need a function that will combine multiple dataframes into one

In [44]:
path='C:/Users/Akis-/OneDrive/Masters/Web Economics/Data/Predictions/basebid_predictions_achilleas/'
teamABaseBidData = pd.read_csv(path+'ridge_basebid.csv')
teamABaseBid=teamABaseBidData[['base_bid']]
#teamABaseBid = teamABaseBid.rename(index=str, columns={"click_proba": "pCTR"})
print(teamABaseBid.head(2))


    base_bid
0  80.350004
1  95.867239


# Build Bid Prices for all models

Need a function that will build a new dataframe conatining bids of all teams by combining the previous two dataframes.

In [45]:
def Function1(resBaseBid,resCTR):
    avCTR = 0.5
    Bids=resBaseBid.values * (resCTR/avCTR)
    #print("Bids:",Bids)
    return Bids

In [57]:
def Function2(reBaseBid,reCTR):
    avCTR = 0.5
    reCTR=reCTR.copy(True)
    reCTR=(((reCTR+0.5)**2)-0.5)
    reCTR[reCTR < 0] = 0
    Bids=reBaseBid.values * (reCTR/avCTR)
    return Bids

In [61]:
def Function3(rBaseBid,rCTR):
    avCTR = 0.5
    rCTR=rCTR.copy(True)
    rCTR[rCTR < 0.5] = 0
    #rCTR=(((rCTR+0.5)**4)-0.5)
    Bids=rBaseBid.values*1.2 * (rCTR/avCTR)
    return Bids

In [63]:
resultsBids=pd.DataFrame(index=range(validation_df.shape[0]))
resultsBids['click']=validation_df['click']
resultsBids['payprice']=validation_df['payprice']

#print(teamABaseBid.head(20))
#print(teamACTR.head(20))
#resultsBids = resultsBids.set_index(teamABaseBid.index)
resultsBids['TeamAFormula1'] = Function1(teamABaseBid,teamACTR).values
resultsBids['TeamAFormula2'] = Function2(teamABaseBid,teamACTR).values
resultsBids['TeamAFormula3'] = Function3(teamABaseBid,teamACTR).values
resultsBids['TeamBFormula1'] = Function1(teamABaseBid,teamBCTR).values
resultsBids['TeamBFormula2'] = Function2(teamABaseBid,teamBCTR).values
resultsBids['TeamBFormula3'] = Function3(teamABaseBid,teamBCTR).values
#print("What:",resultsBids['TeamAFormula1'])

#resultsBids['TeamAFormula2'] = Function2(teamABaseBid,teamACTR)
#resultsBids['TeamAFormula3'] = Function3(teamABaseBid,teamACTR)

print(resultsBids.head(20))

    click  payprice  TeamAFormula1  TeamAFormula2  TeamAFormula3  \
0       0        23      75.681350      71.148330       0.000000   
1       0        75      75.357042      57.040860       0.000000   
2       0        65      79.423925      64.409106       0.000000   
3       0         6      36.382727      27.539590       0.000000   
4       0         5      46.016489      34.831782       0.000000   
5       0        22      37.358419      35.606068       0.000000   
6       0        31      56.270605      45.632842       0.000000   
7       0        20      83.579240      78.573167       0.000000   
8       0        58      39.174854      29.653066       0.000000   
9       0        55      47.593749      39.291408       0.000000   
10      0        88      80.067262      67.254642       0.000000   
11      0        55      42.885943      33.118431       0.000000   
12      0        49      78.761406      75.066988       0.000000   
13      0       110      49.670200      46.69514

In [64]:
print(resultsBids.head(20))

    click  payprice  TeamAFormula1  TeamAFormula2  TeamAFormula3  \
0       0        23      75.681350      71.148330       0.000000   
1       0        75      75.357042      57.040860       0.000000   
2       0        65      79.423925      64.409106       0.000000   
3       0         6      36.382727      27.539590       0.000000   
4       0         5      46.016489      34.831782       0.000000   
5       0        22      37.358419      35.606068       0.000000   
6       0        31      56.270605      45.632842       0.000000   
7       0        20      83.579240      78.573167       0.000000   
8       0        58      39.174854      29.653066       0.000000   
9       0        55      47.593749      39.291408       0.000000   
10      0        88      80.067262      67.254642       0.000000   
11      0        55      42.885943      33.118431       0.000000   
12      0        49      78.761406      75.066988       0.000000   
13      0       110      49.670200      46.69514

# Hold Auction for each

Need a function that will step through the impressions one by one checking who won the impression, whilst keeping track of each player's budget.

In [65]:
print(resultsBids.head())
#Scorer()

   click  payprice  TeamAFormula1  TeamAFormula2  TeamAFormula3  \
0      0        23      75.681350      71.148330            0.0   
1      0        75      75.357042      57.040860            0.0   
2      0        65      79.423925      64.409106            0.0   
3      0         6      36.382727      27.539590            0.0   
4      0         5      46.016489      34.831782            0.0   

   TeamBFormula1  TeamBFormula2  TeamBFormula3  
0            0.0            0.0            0.0  
1            0.0            0.0            0.0  
2            0.0            0.0            0.0  
3            0.0            0.0            0.0  
4            0.0            0.0            0.0  


In [66]:
s = Scorer()
df_summary,df = s.set_df(resultsBids, ['payprice','TeamAFormula1', 'TeamAFormula2', 'TeamAFormula3','TeamBFormula1', 'TeamBFormula2', 'TeamBFormula3'],6250000,200)

# Scores

Summarise the main metrics for each team.

In [67]:
print(df_summary)

   click      team_name    win
0      5  TeamAFormula1  31250
1      6       payprice  31240
2    107  TeamBFormula2  28193
3     35  TeamBFormula3  15946
4      0  TeamAFormula3   3918
5      7  TeamAFormula2  31238
6     15  TeamBFormula1  26280


In [68]:
print(df)

        click  payprice  TeamAFormula1  TeamAFormula2  TeamAFormula3  \
0           0        23      75.681350      71.148330       0.000000   
1           0        75      75.357042      57.040860       0.000000   
2           0        65      79.423925      64.409106       0.000000   
3           0         6      36.382727      27.539590       0.000000   
4           0         5      46.016489      34.831782       0.000000   
5           0        22      37.358419      35.606068       0.000000   
6           0        31      56.270605      45.632842       0.000000   
7           0        20      83.579240      78.573167       0.000000   
8           0        58      39.174854      29.653066       0.000000   
9           0        55      47.593749      39.291408       0.000000   
10          0        88      80.067262      67.254642       0.000000   
11          0        55      42.885943      33.118431       0.000000   
12          0        49      78.761406      75.066988       0.00